# Question to keep in mind: 
1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed?
8. Anything else I should be aware of?

# Acquire the data: 
- Aquire the logs and cohort data from the carriculum_logs

In [1]:
# imports: 
import numpy as np
import pandas as pd 

# environment: 
import env
import os
import wrangle as wr

#warnigns
import warnings
warnings.filterwarnings("ignore")

In [2]:
# url:
url =  f'mysql+pymysql://{env.username}:{env.password}@{env.host}/curriculum_logs'

In [3]:
# sql_query: 
query = '''
SELECT name, start_date, end_date, program_id, date, time, path, user_id, cohort_id, ip
FROM cohorts
LEFT JOIN logs
    on cohorts.id = logs.cohort_id'''

In [4]:
# get the dataframe: 
df = pd.read_sql(query, url)

In [5]:
# look at the data: 
df.head(2)

,name,start_date,end_date,program_id,date,time,path,user_id,cohort_id,ip
0,Arches,2014-02-04,2014-04-22,1,2018-01-31,15:52:32,javascript-i/arrays,11.0,1.0,97.105.19.61
1,Arches,2014-02-04,2014-04-22,1,2018-01-31,15:52:29,javascript-i,11.0,1.0,97.105.19.61


### Function: 

In [6]:
def get_db_url(db, user=env.username, host=env.host, password=env.password):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [7]:
def acquire_logs(): 
    '''
    This function will request the curriculum logs data from sql and return a pandas df
    '''
    sql_query = '''
SELECT name, start_date, end_date, program_id, date, time, path, user_id, cohort_id, ip
FROM cohorts
LEFT JOIN logs
    on cohorts.id = logs.cohort_id'''
    df = pd.read_sql(sql_query, get_db_url('curriculum_logs'))
    return df

In [8]:
#check that it works: 
df = acquire_logs()

In [9]:
df.head()

,name,start_date,end_date,program_id,date,time,path,user_id,cohort_id,ip
0,Arches,2014-02-04,2014-04-22,1,2018-01-31,15:52:32,javascript-i/arrays,11.0,1.0,97.105.19.61
1,Arches,2014-02-04,2014-04-22,1,2018-01-31,15:52:29,javascript-i,11.0,1.0,97.105.19.61
2,Arches,2014-02-04,2014-04-22,1,2018-01-31,14:59:38,javascript-i/arrays,11.0,1.0,97.105.19.61
3,Arches,2014-02-04,2014-04-22,1,2018-01-31,14:32:22,javascript-i,11.0,1.0,97.105.19.61
4,Arches,2014-02-04,2014-04-22,1,2018-01-31,14:32:19,javascript-i/arrays,11.0,1.0,97.105.19.61


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847336 entries, 0 to 847335
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   name        847336 non-null  object 
 1   start_date  847336 non-null  object 
 2   end_date    847336 non-null  object 
 3   program_id  847336 non-null  int64  
 4   date        847330 non-null  object 
 5   time        847330 non-null  object 
 6   path        847329 non-null  object 
 7   user_id     847330 non-null  float64
 8   cohort_id   847330 non-null  float64
 9   ip          847330 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 64.6+ MB


In [11]:
# read the data into a csv: 
def get_logs():
    '''
    This functino will read in curriculum logs and write it into a csv file if a local file does not exist. 
    Returns a dataframe.
    '''
    if os.path.isfile('curriculum.csv'):
        
        # If csv file exists, read in data from csv file.
        df = pd.read_csv('curriculum.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame.
        df = acquire_logs()
        
        # Write DataFrame to a csv file.
        df.to_csv('curriculum.csv')
        
    return df   

In [12]:
df = acquire_logs()

In [13]:
df = get_logs()

# Data Preperation: Non time series: 

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847336 entries, 0 to 847335
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   name        847336 non-null  object 
 1   start_date  847336 non-null  object 
 2   end_date    847336 non-null  object 
 3   program_id  847336 non-null  int64  
 4   date        847330 non-null  object 
 5   time        847330 non-null  object 
 6   path        847329 non-null  object 
 7   user_id     847330 non-null  float64
 8   cohort_id   847330 non-null  float64
 9   ip          847330 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 71.1+ MB


In [15]:
#look at the null values: 
df.isnull().sum().sort_values(ascending = False)

path          7
date          6
time          6
user_id       6
cohort_id     6
ip            6
name          0
start_date    0
end_date      0
program_id    0
dtype: int64

In [16]:
# function to create a datatable of nulls and the percentage that is missing
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    pct_missing = (num_missing / rows) 
    cols_missing = pd.DataFrame({'number_missing_rows': num_missing, 'percent_rows_missing': pct_missing})
    return cols_missing

In [17]:
# how big of a percent is this? 
nulls = nulls_by_col(df)
nulls.sort_values(by = 'percent_rows_missing', ascending = False)

,number_missing_rows,percent_rows_missing
path,7,0.000008
date,6,0.000007
time,6,0.000007
user_id,6,0.000007
cohort_id,6,0.000007
ip,6,0.000007
name,0,0.000000
start_date,0,0.000000
end_date,0,0.000000
program_id,0,0.000000


In [18]:
# the nulls are so little: 
df = df.dropna()

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847329 entries, 0 to 847335
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   name        847329 non-null  object 
 1   start_date  847329 non-null  object 
 2   end_date    847329 non-null  object 
 3   program_id  847329 non-null  int64  
 4   date        847329 non-null  object 
 5   time        847329 non-null  object 
 6   path        847329 non-null  object 
 7   user_id     847329 non-null  float64
 8   cohort_id   847329 non-null  float64
 9   ip          847329 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 71.1+ MB


In [20]:
# change date + time into datetime
df['datetime'] = df['date'] + " " + df['time']

In [21]:
df.columns

Index(['name', 'start_date', 'end_date', 'program_id', 'date', 'time', 'path',
       'user_id', 'cohort_id', 'ip', 'datetime'],
      dtype='object')

In [22]:
df = df.drop(columns = ['date','time'])

In [23]:
df.columns

Index(['name', 'start_date', 'end_date', 'program_id', 'path', 'user_id',
       'cohort_id', 'ip', 'datetime'],
      dtype='object')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847329 entries, 0 to 847335
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   name        847329 non-null  object 
 1   start_date  847329 non-null  object 
 2   end_date    847329 non-null  object 
 3   program_id  847329 non-null  int64  
 4   path        847329 non-null  object 
 5   user_id     847329 non-null  float64
 6   cohort_id   847329 non-null  float64
 7   ip          847329 non-null  object 
 8   datetime    847329 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 64.6+ MB


### Function: 

In [25]:
def prep_logs_plain():
    '''
    This function will prepare the dataframe by: 
    - Remoing null values
    - Droping unnecessary columns
    - create a datetime that can be used for an index
    '''
    df = acquire_logs()
    # drop nulls
    df = df.dropna()
    # create datetime: 
    df['datetime'] = df['date'] + " " + df['time']
    # set the date time to a dataframe: 
    df.datetime = pd.to_datetime(df.datetime)
    # drop columns: 
    df = df.drop(columns = ['date','time'])
    
    return df

In [26]:
df = acquire_logs()

In [27]:
df.head()

,name,start_date,end_date,program_id,date,time,path,user_id,cohort_id,ip
0,Arches,2014-02-04,2014-04-22,1,2018-01-31,15:52:32,javascript-i/arrays,11.0,1.0,97.105.19.61
1,Arches,2014-02-04,2014-04-22,1,2018-01-31,15:52:29,javascript-i,11.0,1.0,97.105.19.61
2,Arches,2014-02-04,2014-04-22,1,2018-01-31,14:59:38,javascript-i/arrays,11.0,1.0,97.105.19.61
3,Arches,2014-02-04,2014-04-22,1,2018-01-31,14:32:22,javascript-i,11.0,1.0,97.105.19.61
4,Arches,2014-02-04,2014-04-22,1,2018-01-31,14:32:19,javascript-i/arrays,11.0,1.0,97.105.19.61


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847336 entries, 0 to 847335
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   name        847336 non-null  object 
 1   start_date  847336 non-null  object 
 2   end_date    847336 non-null  object 
 3   program_id  847336 non-null  int64  
 4   date        847330 non-null  object 
 5   time        847330 non-null  object 
 6   path        847329 non-null  object 
 7   user_id     847330 non-null  float64
 8   cohort_id   847330 non-null  float64
 9   ip          847330 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 64.6+ MB


# Prepare data: Time Series data: 


In [29]:
# set the date time to a dataframe: 
df.datetime = pd.to_datetime(df.datetime)

AttributeError: 'DataFrame' object has no attribute 'datetime'

In [ ]:
# set the index to be the date: 
df = df.set_index(df.datetime)

In [ ]:
# want the weekday: 
df['weelday'] = df.index.day_name()

In [ ]:
# want the month: 
df['month'] = df.index.month_name()

### Function

In [ ]:
def prep_logs_time():
    '''
    This function will prepare the dataframe by: 
    - Remoing null values
    - Droping unnecessary columns
    - create a datetime that can be used for an index
    - make the index the datetime
    '''
    df = acquire_logs()
    # drop nulls
    df = df.dropna()
    # create datetime: 
    df['datetime'] = df['date'] + " " + df['time']
    # set the date time to a dataframe: 
    df.datetime = pd.to_datetime(df.datetime)
    # drop columns: 
    df = df.drop(columns = ['date','time'])
    
    # set the index to datetime: 
    df = df.set_index(df.datetime)
    # create the weekday: 
    df['weelday'] = df.index.day_name()
    # want the month: 
    df['month'] = df.index.month_name()
    
    return df
